In [ ]:
from __future__ import annotations
import datetime as dt
from typing import NamedTuple
from collections.abc import Iterable
from random import random
import sys

from micro_namedtuple_sqlite_persister.persister import Engine
from micro_namedtuple_sqlite_persister.adaptconvert import register_standard_adaptconverters
register_standard_adaptconverters()

# Models

In [ ]:
class MyModel(NamedTuple):
    id: int | None
    name: str
    date: dt.datetime
    score: float | None

Connect to the database and create tables with an `Engine`

In [ ]:
engine = Engine(":memory:")
# engine = Engine("example.db")
engine.connection.set_trace_callback(lambda sql: print(sql, file=sys.stderr)) # echo SQL
engine.ensure_table_created(MyModel)
engine.connection # just the real connection object

# Basic CRUD

## Insert row

In [ ]:
row = MyModel(None, "Bart", dt.datetime.now(), 6.5)
row = engine.save(row)
engine.connection.commit()
row

## Find row by id

In [ ]:
engine.find(MyModel, row.id)

## Find row by Field

In [ ]:
engine.find_by(MyModel, name= "Bart")

## Update row

In [ ]:
engine.save(row._replace(score=78.9))
engine.connection.commit()

## Delete row

by id

In [ ]:
row2 = engine.save(MyModel(None, "foo", dt.datetime.now(), 6.5))

engine.delete(MyModel, row2.id)
engine.connection.commit()

by instance

In [ ]:
row3 = engine.save(MyModel(None, "bar", dt.datetime.now(), 9.5))

engine.delete(row3)
engine.connection.commit()

# Foreign Keys Relationships
Models can be related by using a model as a field type in another model.

In [ ]:
class Band(NamedTuple):
    id: int | None
    name: str
    active: bool

class BandMember(NamedTuple):
    id: int | None
    band: Band
    name: str
    instrument: str

engine.ensure_table_created(Band)
engine.ensure_table_created(BandMember)

## Save deeply

When saving a model with related models, you can use the `deep=True` argument to recursively insert/update/get related models.

Recursively insert/update/get related models. Rows without an id will be inserted, rows with an id will be updated.

In [ ]:
beatles = Band(None, "The Band", True)
paul = BandMember(None, beatles, "Paul McCartney", "Bass")

# saves both the band and the member
paul = engine.save(paul, deep=True)
engine.connection.commit()

## Recursive loading
Related models are loaded recursively.

You can control/disable this by making view models that exclude the related models.

In [ ]:
singer = engine.find(BandMember, paul.id)
display(singer)

# Note how the `band` field gets pulled in
print(singer.name)
print(singer.band.name)

In [ ]:
class League(NamedTuple):
    id: int | None
    leaguename: str

class Team(NamedTuple):
    id: int | None
    teamname: str
    league: League

class Athlete(NamedTuple):
    id: int | None
    name: str
    team: Team


engine.ensure_table_created(League)
engine.ensure_table_created(Team)
engine.ensure_table_created(Athlete)

# Insert dummy data
leagues = [
    engine.save(League(None, "Big")),
    engine.save(League(None, "Small")),
    ]
teams = [
    red:=engine.save(Team(None, "Red", leagues[0])),
    engine.save(Team(None, "Ramble", leagues[1])),
    engine.save(Team(None, "Blue", leagues[0])),
    engine.save(Team(None, "Green", leagues[1])),
    ]
players = [
    alice:=engine.save(Athlete(None, "Alice", teams[0])),
    engine.save(Athlete(None, "Bob", teams[0])),
    engine.save(Athlete(None, "Charlie", teams[1])),
    engine.save(Athlete(None, "Dave", teams[2])),
    engine.save(Athlete(None, "Beth", teams[3])),
    engine.save(Athlete(None, "Frank", teams[2])),
]
engine.connection.commit()

In [ ]:
engine.find(Athlete, alice.id).team.league.leaguename

## Alternate Models
Create a model that queries a subset or alternate form of the data, for example pulling in a foreign key as an int id instead of the full `Model` instance.

The name of the table comes before a '_'

In [ ]:
class Team_NameOnly(NamedTuple):
    id: int | None
    teamname: str

engine.find(Team_NameOnly, alice.team.id)

In [ ]:
class Athlete_TeamAsIntId(NamedTuple):
    id: int | None
    name: str
    team: int

engine.find(Athlete_TeamAsIntId, alice.id)

## Table-less models and the `Any` type
Models can also be completely table-less, for example to represent a view or a query result. These models do not require an id field. They also have a special field type available, `Any`, which can be used to represent any type of data. This is particularly useful for dynamic or polymorphic data structures where the exact type may not be known until runtime.



In [ ]:
from typing import Any

class TableInfo(NamedTuple):
    cid: int
    name: str
    type: str
    notnull: int
    dflt_value: Any # `Any` will return raw value matching python's bare sqlite3, without conversion
    pk: int

sql = f"PRAGMA table_info({Athlete.__name__})"

cols = engine.query(TableInfo, sql).fetchall()
for col in cols:
    print(f"{col.cid:2d} {col.name:10s} {col.type:10s} {str(col.dflt_value or 'None'):10s}")

# Querying

The interface for querying is very simple, you can use any arbitrary SQL directly.

```python
engine.query(Model, sql)
```

In [ ]:
class AverageScoreResults(NamedTuple):
    avg_score: float
    scorecount: int

sql = 'SELECT avg(score), count(*) FROM MyModel'

result = engine.query(AverageScoreResults, sql).fetchone()
assert result is not None

print(f'The table has {result.scorecount} rows, with and average of {result.avg_score:0.2f}')

`select` is a simple yet powerful way to retrieve Models from the database.

The most simple case selects all rows from a table.

`select` returns a tuple of `(Model, query)`, matching the `engine.query` interface.


In [ ]:
from micro_namedtuple_sqlite_persister.query import select

M, q = select(Athlete)

for player in engine.query(M, q).fetchall():
    print(player)


For brevity, inline `select(Model)` and `*`splat the `(M,q)` tuple right into `engine.query`.

In [ ]:
for player in engine.query(*select(Athlete)).fetchall():
    print(player)

The select query can easily be modified to add `WHERE` clauses.

> **Note:** These `def`s never actually run, but instead are analyzed through reflection to build the query.

```python

In [ ]:
@select(Athlete)
def athletes_named_beth():
    return f"WHERE {Athlete.name} LIKE 'B%'"

for player in engine.query(*athletes_named_beth()).fetchall():
    print(player)

Joins are automatically determined by the field path (e.g. `Athlete.team.name`). The `Model` class referenced by the `select` must always be the root of your field path.

In [ ]:
# a single join
@select(Athlete)
def athletes_on_red_team():
    return f"WHERE {Athlete.team.teamname} = 'Red'"

# two joins required
@select(Athlete)
def athletes_in_big_leagues():
    return f"WHERE {Athlete.team.league.leaguename} = 'Big'"


for player in engine.query(*athletes_on_red_team()).fetchall():
    print(player)

for player in engine.query(*athletes_in_big_leagues()).fetchall():
    print(player)

# Querys requiring backrefs

Eventually we will support backrefs:

```python
class League(NamedTuple):
    id: int | None
    leaguename: str
    teams: List[Team] # backref will be something like this
```

And then you could do:

```python
@select(League)
def leagues_with_big_teams():
    return f"WHERE {League.teams.teamname} = 'Big'"
```

For now just fallback to raw queries: 


In [ ]:
sql = 'SELECT * FROM League JOIN Team ON Team.league = League.id WHERE Team.teamname = "Big"'
result = engine.query(League, sql).fetchone()


## Query Parameters
The decorated function can also take parameters which will be return as a third element in the tuple.

In [ ]:
@select(Athlete)
def athletes_in_league(league: str):
    return f"WHERE {Athlete.team.league.leaguename} = {league}"

M, q, p = athletes_in_league('Big')

print("Our query:")
print(q)
print(p)
print()

for player in engine.query(M, q, p).fetchall():
    print(player)

# or the same, but more concisely
for player in engine.query(*athletes_in_league('Small')).fetchall():
    print(player)

## SQLite3 Cursor
Notice that query returns a real `sqlite3.Cursor`, you can use it to `fetchall`, `fetchone`, `fetchmany`, etc.

The only thing we do is set the `Cursor.row_factory` to return Model instances, and stub the static typehints in for them as well.

In [ ]:
engine.query(*select(Athlete)).fetchone()

# Persisting native python collections
You can persist any list or dict that recusively serializes to valid JSON using the `json.dumps'and `json.loads` methods. Enums will be supported in the future.

In [ ]:
class JsonExample(NamedTuple):
    id: int | None
    names: dict

engine.ensure_table_created(JsonExample)
names = {"Alice": 1, "Bob": 2, "Charlie": 3}
row = engine.save(JsonExample(None, names))

engine.find(JsonExample, row.id)

## SQLite3 supports JSON extensions

In [ ]:
class Character(NamedTuple):
    id: int | None
    name: str
    stats: dict

engine.ensure_table_created(Character)

engine.save(Character(None, 'Harbel', {'spell': 'Fireball', 'level': 3}))
engine.save(Character(None, 'Quenswen', {'spell': 'Waterspout', 'level': 27}))
engine.save(Character(None, 'Ruthbag', {'spell': 'Fireball', 'level': 12}))

@select(Character)
def get_fireball_characters():
    f"WHERE {Character.stats} ->> '$.spell' = 'Fireball'"

for c in engine.query(*get_fireball_characters()).fetchall():
    print(f"{c.name} has a fireball at level {c.stats['level']}")

# Persisting Custom Types: Adapt/Convert

In [ ]:
class PhoneNumber:
    def __init__(self, area_code: int, number: int):
        self.area_code = area_code
        self.number = number

    def __repr__(self):
        return f"PhoneNumber({self.area_code}, {self.number})"

def adapt_phonenumber(obj: PhoneNumber) -> bytes:
    return f"({obj.area_code}) {obj.number}".encode()

def convert_phonenumber(data: bytes) -> PhoneNumber:
    area_code, number = data.decode().split(" ", 1)
    return PhoneNumber(int(area_code.strip("()")), int(number))
from micro_namedtuple_sqlite_persister.adaptconvert import register_adapt_convert, AdaptConvertTypeAlreadyRegistered


register_adapt_convert(PhoneNumber, adapt_phonenumber, convert_phonenumber, overwrite=True)


class LibraryMember(NamedTuple):
    id: int | None
    name: str
    pnumber: PhoneNumber

engine.ensure_table_created(LibraryMember)
row2 = LibraryMember(None, "Alice", PhoneNumber(123, 4567890))

row2 = engine.save(row2)
engine.connection.commit()

engine.find(LibraryMember, row2.id)

There is a special helper function for the common case of pickleable types, like `pandas.DataFrame`, `numpy.ndarray`, etc.

In [ ]:
import pandas as pd

from micro_namedtuple_sqlite_persister.adaptconvert import register_pickleable_adapt_convert

class DataAnalysis(NamedTuple):
    id: int | None
    name: str
    df: pd.DataFrame

register_pickleable_adapt_convert(pd.DataFrame)
engine.ensure_table_created(DataAnalysis)

df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
row = engine.save(DataAnalysis(None, "foo", df))

engine.find(DataAnalysis, row.id).df


# Performance scenarios
Every call to insert real full trip to the db. The data is ready to be queried immediately, in SQLAlchemy parlance, 'flushed'. Committig ends the implicit transaction and ensures that the data is persisted to disk. Data is then avialable to other connections e.g. other worker processes

Because the db and app share a process, the performance is good enough that you can basically ignore the N+1 problem. This also simplifies implementation of this library, no need to track session etc. It also simplifies your app as data is syncronized immediately with the database, thus eliminates the need for a stateful cache, a source off many bugs and complexity.

In [ ]:
engine.connection.set_trace_callback(None) # disable echo SQL

## Insert many (17,000 rows)

In [ ]:
rows = [MyModel(None, "foo", dt.datetime.now(), random()*100) for _ in range(17000)]

In [ ]:
for r in rows:
    engine.save(r)

engine.connection.commit()

## Update many (17,000 rows)

In [ ]:
rows = [row._replace(date=dt.datetime.now(), score=random()*100) for row in rows]

In [ ]:
for r in rows:
    engine.save(r)

engine.connection.commit()

## Query many

In [ ]:
def print_30_per_line(ss: Iterable[str]):
    for i,s in enumerate(ss, 1):
        print(s, end=" ")
        if i % 30 == 0:
            print()
    print()

@select(MyModel)
def high_scores():
    return f"WHERE {MyModel.score} > 95.7"

rows = engine.query(*high_scores()).fetchall()
print_30_per_line(f"{r.score:5.1f}" for r in rows)

## Giant Recursive BOM

In [ ]:
class BOM(NamedTuple):
    id: int | None
    name: str
    value: float
    child_a: BOM | None
    child_b: BOM | None

engine.ensure_table_created(BOM)

from random import random, choice
node_count = 0
def generate_node_name_node(depth: int) -> str:
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    return f"{choice(alphabet)}{choice(alphabet)}{choice(alphabet)}{depth:05d}_{node_count}"


# create a giant BOM, of 15 levels deep
def create_bom(depth: int) -> BOM:
    global node_count
    node_count += 1

    if depth == 1:
        child_a = None
        child_b = None
    else:
        child_a = create_bom(depth-1)
        child_b = create_bom(depth-1)

    return BOM(None, generate_node_name_node(depth), random()*1000 - 500, child_a, child_b)

root = create_bom(13)
print(f"Created a BOM with {node_count} nodes")

In [ ]:
inserted_root = engine.save(root, deep=True)
engine.connection.commit()

print(f"Inserted BOM with id: {inserted_root.id}")

In [ ]:
recovered_root = engine.find(BOM, inserted_root.id, deep=True)

def count_nodes(node: BOM | None) -> int:
    if node is None:
        return 0
    return 1 + count_nodes(node.child_a) + count_nodes(node.child_b)

print(f"Recovered BOM with {count_nodes(recovered_root)} nodes")

In [ ]:
import math

import matplotlib.pyplot as plt
import networkx as nx

def add_nodes_edges(G: nx.Graph, node: BOM | None):
    if node is None:
        return

    G.add_node(node.id, label=node.name)
    if node.child_a is not None:
        G.add_edge(node.id, node.child_a.id)
        add_nodes_edges(G, node.child_a)

    if node.child_b is not None:
        G.add_edge(node.id, node.child_b.id)
        add_nodes_edges(G, node.child_b)

def hierarchical_tree_layout(G, root_node):
    pos = {}

    # Build adjacency list from the graph
    adj = {node: list(G.neighbors(node)) for node in G.nodes()}

    # BFS to determine levels and children
    from collections import deque
    queue = deque([(root_node, 0)])
    visited = {root_node}
    levels = {}
    children = {node: [] for node in G.nodes()}

    while queue:
        node, level = queue.popleft()
        levels[node] = level

        for neighbor in adj[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                children[node].append(neighbor)
                queue.append((neighbor, level + 1))

    # Position nodes level by level
    def position_subtree(node, level, angle_start, angle_end):
        # Position current node
        if level == 0:
            pos[node] = (0, 0)  # Root at center
        else:
            angle = (angle_start + angle_end)
            radius = level * 0.8  # Increase radius per level
            x = radius * math.cos(angle)
            y = radius * math.sin(angle)
            pos[node] = (x, y)

        # Position children
        kids = children[node]
        if kids:
            angle_span = min(angle_end - angle_start, 2 * math.pi / max(1, len(kids)))
            angle_per_child = angle_span / len(kids)

            for i, child in enumerate(kids):
                child_angle_start = angle_start + i * angle_per_child
                child_angle_end = child_angle_start + angle_per_child
                position_subtree(child, level + 1, child_angle_start, child_angle_end)

    # Start positioning from root
    position_subtree(root_node, 0, 0, 2 * math.pi)
    return pos


G = nx.Graph()
add_nodes_edges(G, recovered_root)

plt.figure(figsize=(10, 10))
nx.draw(G, hierarchical_tree_layout(G, recovered_root.id),
    node_size=6, width=0.2, node_color="blue",
    with_labels=node_count<1200,
    labels=nx.get_node_attributes(G, "label"),
    )
plt.show()

# Meta
Meta is a global cache of model metadata. It is global, rather then per-engine, because sqlite3 adapters and converters are global, and we must avoid a having conflicting adapter/converters/models.

Debug out internal meta

In [ ]:
from micro_namedtuple_sqlite_persister.model import _meta
for k,v in _meta.items():
    print(f"{k}: {v}")